<a href="https://colab.research.google.com/github/rushikesh-mallanath/draw/blob/main/spacymodeltraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -U spacy -q

In [2]:
!python -m spacy info

2023-09-21 08:01:21.389006: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 08:01:22.220324: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

============================== Info about spaCy ==============================

spaCy version    3.6.1                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-5.15.120+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        en_core_web_sm (3.6.0)        



In [3]:
!python -m spacy download en_core_web_lg

2023-09-21 08:02:49.960950: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 08:02:50.770183: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [4]:
!python -m spacy info

2023-09-21 08:04:49.840713: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 08:04:51.219189: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

============================== Info about spaCy ==============================

spaCy version    3.6.1                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-5.15.120+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        en_core_web_lg (3.6.0), en_core_web_sm (3.6.0)



In [5]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en")
db = DocBin()

In [6]:
import json
f = open('training_data.json')
TRAIN_DATA = json.load(f)

In [12]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
  doc = nlp.make_doc(text)
  ents = []
  for start, end, label in annot["entities"]:
    span = doc.char_span(start, end, label = label, alignment_mode="contract")
    if span is None:
      print("Skipping entity")
    else:
      ents.append(span)
  doc.ents = ents
  db.add(doc)

db.to_disk("/content/training_data.spacy")

100%|██████████| 3/3 [00:00<00:00, 44.41it/s]


In [11]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize accuracy

2023-09-21 08:22:24.658209: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [15]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

2023-09-21 09:39:09.233530: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00   1302.23    0.00    0.00    0.00    0.00
 33     200       5655.71  18075.42   89.09   89.09   89.09    0.89
 66     400         12.48    132.06  100.00  100.00  100.00    1.00
100     600          0.00      0.00  100.00  100.00  100.00    1.00
133     800          0.00      0.00  100.00  100.00  100.00    1.00
166    1000          0.00      0.00  100.00  100.00  100.00    1.00
200    1200        482.92    675.17  100.00  100

In [16]:
nlp = spacy.load("/content/model-best")

In [17]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 19.0 MB/s eta 0:00:00


In [32]:
import sys, fitz
import os

fname = '/content/resume.pdf'
mydoc = fitz.open(fname)

In [33]:
text = " "
for page in mydoc:
    text = text + str(page.get_text())

In [34]:
text

' Omkar Pathak\nSOFTWARE ENGINEER · FULL STACK PYTHON DEVELOPER\nPune, Maharashtra, India\n\uf10b (+91) 8087996634\n|\n\uf0e0 omkarpathak27@gmail.com\n|\n\uf015 www.omkarpathak.in\n|\n\uf092 OmkarPathak\n|\n\uf08c omkar-pathak-94473811b\n“Make the change that you want to see in the world.”\nExperience\nSchlumberger\nPune, Maharashtra, India\nDATA ENGINEER\nJuly 2018 - Present\n• Responsible for implementing and managing an end-to-end CI/CD Pipeline with custom validations for Informatica migrations which\nbrought migration time to 1.5 hours from 9 hours without any manual intervention\n• Enhancing, auditing and maintaining custom data ingestion framework that ingest around 1TB of data each day to over 70 business\nunits\n• Working with L3 developer team to ensure the discussed Scrum PBI’s are delivered on time for data ingestions\n• Planning and Executing QA and Production Release Cycle activities\nTruso\nPune, Maharashtra, India\nFULL STACK DEVELOPER INTERN\nJune 2018 - July 2018\n• C

In [35]:
mydoc = nlp(text)
for ent in mydoc.ents:
    print(ent.text, " - ", ent.label_)

July 2018 - Present
• Responsible for implementing and managing an end-to-end CI/CD Pipeline with custom validations for Informatica migrations which
brought migration time to 1.5 hours from  -  EXPERIENCE
June 2018 - July 2018
• Created RESTful apis
• Tried my hands on Angular 5/6
• Was responsible for Django backend development
Propeluss
Pune, Maharashtra, India
DATA ENGINEERING INTERN
October 2017 - January 2018
• Wrote various automation scripts to scrape data from various websites.
• Applied Natural Language Processing to articles scraped  -  EXPERIENCE
July 2017 - September 2017
• Published 4 articles for the topics such as Data Structures and Algorithms and Python
Softtestlab Technologies
Pune, Maharashtra, India
WEB DEVELOPER INTERN
June 2017 - July 2017
• Was responsible for creating an internal project for the company using PHP and Laravel for testing purposes
• Worked on a live project for creating closure reports using PHP and Excel
Projects
Pyresparser
API/Python Package
P